In [1]:
# ============================================================================
# NHIỆM VỤ 4: MERGE DỮ LIỆU
# ============================================================================

import pandas as pd
import numpy as np
from datetime import datetime

# ------------------------------
# 1. ĐỌC 3 FILE DỮ LIỆU ĐÃ LÀM SẠCH (từ Nhiệm vụ 1)
# ------------------------------
print("="*60)
print("NHIỆM VỤ 4: MERGE DỮ LIỆU")
print("="*60)

# Đọc dữ liệu đã làm sạch (giả sử đã có từ Task 1)
print("\n1. Đọc dữ liệu đã làm sạch từ Task 1...")

# Nếu bạn chưa có file sạch, tôi sẽ tạo code làm sạch cơ bản ngay tại đây
def clean_and_load_data():
    """
    Hàm này làm sạch dữ liệu thô nếu chưa có file sạch từ Task 1
    """
    print("Chưa có file sạch, đang thực hiện làm sạch cơ bản...")
    
    # 1. Đọc dữ liệu thô
    hotel_raw = pd.read_csv("hotel_info.csv")
    booking_raw = pd.read_csv("booking_records.csv")
    review_raw = pd.read_csv("guest_review.csv")
    
    # 2. Làm sạch hotel_info.csv
    hotel_clean = hotel_raw.copy()
    
    # Chuẩn hóa hotel_id (viết hoa, bỏ khoảng trắng)
    hotel_clean['hotel_id'] = hotel_clean['hotel_id'].astype(str).str.strip().str.upper()
    
    # Chuẩn hóa hotel_name
    hotel_clean['hotel_name'] = hotel_clean['hotel_name'].str.strip()
    
    # Chuẩn hóa city
    city_mapping = {
        'ha noi': 'Hà Nội',
        'Ha Noi': 'Hà Nội',
        'Hà Nội': 'Hà Nội',
        'Đà Nẵng': 'Đà Nẵng',
        'Da Nang': 'Đà Nẵng',
        'Da Nang ': 'Đà Nẵng',
        'TP.HCM': 'TP.HCM',
        'Tp. HCM': 'TP.HCM',
        'TP.HCM ': 'TP.HCM',
        'TP HCM': 'TP.HCM'
    }
    hotel_clean['city'] = hotel_clean['city'].str.strip().replace(city_mapping)
    
    # Chuẩn hóa room_type
    hotel_clean['room_type'] = hotel_clean['room_type'].str.strip().str.title()
    
    # Làm sạch base_price
    def clean_price(price):
        if isinstance(price, str):
            # Loại bỏ ký tự tiền tệ, dấu chấm
            price = price.replace('đ', '').replace('k', '000').replace('.', '').replace(',', '')
            # Loại bỏ khoảng trắng
            price = price.strip()
            try:
                return int(price)
            except:
                return np.nan
        return price
    
    hotel_clean['base_price'] = hotel_clean['base_price'].apply(clean_price)
    
    # 3. Làm sạch booking_records.csv
    booking_clean = booking_raw.copy()
    
    # Chuẩn hóa hotel_id
    booking_clean['hotel_id'] = booking_clean['hotel_id'].astype(str).str.strip().str.upper()
    
    # Chuẩn hóa checkin_date
    def parse_date(date_str):
        try:
            if '/' in str(date_str):
                return datetime.strptime(str(date_str), '%d/%m/%Y')
            else:
                return datetime.strptime(str(date_str), '%Y-%m-%d')
        except:
            return pd.NaT
    
    booking_clean['checkin_date'] = booking_clean['checkin_date'].apply(parse_date)
    
    # Làm sạch nights
    def clean_nights(night):
        if isinstance(night, str):
            night = night.strip().replace('n', '').replace('N', '')
        try:
            night_val = int(float(night))
            return night_val if night_val > 0 else np.nan
        except:
            return np.nan
    
    booking_clean['nights'] = booking_clean['nights'].apply(clean_nights)
    
    # Làm sạch num_guests
    def clean_guests(guest):
        if isinstance(guest, str):
            guest = guest.strip().replace('p', '').replace('P', '')
        try:
            guest_val = int(float(guest))
            return guest_val if guest_val > 0 else np.nan
        except:
            return np.nan
    
    booking_clean['num_guests'] = booking_clean['num_guests'].apply(clean_guests)
    
    # 4. Làm sạch guest_review.csv
    review_clean = review_raw.copy()
    
    # Làm sạch rating
    def clean_rating(rating):
        if isinstance(rating, str):
            rating = rating.strip()
            # Xử lý rating có dấu * như '5*'
            rating = rating.replace('*', '')
        try:
            rating_val = float(rating)
            # Chỉ giữ rating từ 1-5
            if 1 <= rating_val <= 5:
                return rating_val
            else:
                return np.nan
        except:
            return np.nan
    
    review_clean['rating'] = review_clean['rating'].apply(clean_rating)
    
    # Làm sạch comment (thay thế ký tự đặc biệt)
    def clean_comment(comment):
        if isinstance(comment, str):
            comment = comment.strip()
            # Thay thế ký tự đặc biệt không có ý nghĩa
            if comment in ['', ' ', '!@#$']:
                return np.nan
            # Chuẩn hóa một số từ viết tắt
            comment = comment.replace('Tot', 'Tốt')
            return comment
        return comment
    
    review_clean['comment'] = review_clean['comment'].apply(clean_comment)
    
    # Xóa các booking_id trùng lặp (giữ bản ghi đầu tiên)
    review_clean = review_clean.drop_duplicates(subset=['booking_id'], keep='first')
    
    return hotel_clean, booking_clean, review_clean

# Thử đọc file sạch, nếu không có thì làm sạch
try:
    hotel_clean = pd.read_csv("data_clean/hotel_clean.csv")
    booking_clean = pd.read_csv("data_clean/booking_clean.csv") 
    review_clean = pd.read_csv("data_clean/review_clean.csv")
    print("✓ Đã đọc file sạch từ Task 1")
except FileNotFoundError:
    print("⚠ Không tìm thấy file sạch, đang thực hiện làm sạch...")
    hotel_clean, booking_clean, review_clean = clean_and_load_data()
    print("✓ Đã hoàn thành làm sạch dữ liệu cơ bản")

print(f"\nKích thước dữ liệu:")
print(f"- hotel_clean: {hotel_clean.shape}")
print(f"- booking_clean: {booking_clean.shape}")
print(f"- review_clean: {review_clean.shape}")

# ------------------------------
# 2. MERGE DỮ LIỆU
# ------------------------------
print("\n" + "-"*60)
print("2. Thực hiện merge dữ liệu")
print("-"*60)

# Merge 1: hotel_info với booking_records
print("\na) Merge hotel_info với booking_records (theo hotel_id)...")
merged_step1 = pd.merge(
    booking_clean,
    hotel_clean,
    on='hotel_id',
    how='left',  # Giữ tất cả booking, kể cả không có thông tin khách sạn
    suffixes=('_booking', '_hotel')
)

print(f"   Kích thước sau merge 1: {merged_step1.shape}")
print(f"   Số booking không khớp hotel_id: {(merged_step1['hotel_name'].isna()).sum()}")

# Merge 2: Kết quả trên với guest_review
print("\nb) Merge thêm với guest_review (theo booking_id)...")
final_merged = pd.merge(
    merged_step1,
    review_clean,
    on='booking_id',
    how='left',  # Giữ tất cả booking, kể cả không có đánh giá
    suffixes=('', '_review')
)

print(f"   Kích thước sau merge 2: {final_merged.shape}")
print(f"   Số booking không có đánh giá: {(final_merged['rating'].isna()).sum()}")

# ------------------------------
# 3. PHÂN TÍCH DỮ LIỆU MERGE
# ------------------------------
print("\n" + "-"*60)
print("3. Phân tích dữ liệu sau merge")
print("-"*60)

# a. Thông tin chung
print("\na) Thông tin chung về dữ liệu merge:")
print(f"   - Tổng số bản ghi: {final_merged.shape[0]}")
print(f"   - Số cột: {final_merged.shape[1]}")
print(f"   - Các cột: {', '.join(final_merged.columns.tolist())}")

# b. Kiểm tra dữ liệu thiếu
print("\nb) Kiểm tra dữ liệu thiếu (Missing Values):")
missing_data = final_merged.isnull().sum()
missing_percent = (missing_data / len(final_merged)) * 100
missing_df = pd.DataFrame({
    'Số lượng thiếu': missing_data,
    'Phần trăm (%)': missing_percent.round(2)
})
print(missing_df[missing_df['Số lượng thiếu'] > 0])

# c. Phát hiện booking không khớp
print("\nc) Phát hiện các booking có vấn đề:")

# Booking không có thông tin khách sạn
no_hotel_info = final_merged[final_merged['hotel_name'].isna()]
print(f"   - Booking không khớp với khách sạn: {len(no_hotel_info)}")

# Booking không có đánh giá
no_review = final_merged[final_merged['rating'].isna()]
print(f"   - Booking không có đánh giá: {len(no_review)}")

# Booking có nights hoặc num_guests bị thiếu
invalid_booking = final_merged[(final_merged['nights'].isna()) | (final_merged['num_guests'].isna())]
print(f"   - Booking có nights hoặc num_guests không hợp lệ: {len(invalid_booking)}")

# d. Hiển thị mẫu dữ liệu
print("\nd) 5 dòng đầu tiên của dữ liệu merge:")
print(final_merged.head())

# ------------------------------
# 4. XUẤT FILE DỮ LIỆU HOÀN CHỈNH
# ------------------------------
print("\n" + "-"*60)
print("4. Xuất file dữ liệu hoàn chỉnh")
print("-"*60)

# Tạo thư mục nếu chưa có
import os
os.makedirs("task04_merge", exist_ok=True)
os.makedirs("data_clean", exist_ok=True)

# Xuất file merge
output_path = "task04_merge/merged_complete.csv"
final_merged.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"✓ Đã xuất file dữ liệu merge: {output_path}")
print(f"✓ Kích thước file: {final_merged.shape[0]} dòng × {final_merged.shape[1]} cột")

# Xuất file sạch nếu chưa có
if not os.path.exists("data_clean/hotel_clean.csv"):
    hotel_clean.to_csv("data_clean/hotel_clean.csv", index=False, encoding='utf-8-sig')
    booking_clean.to_csv("data_clean/booking_clean.csv", index=False, encoding='utf-8-sig')
    review_clean.to_csv("data_clean/review_clean.csv", index=False, encoding='utf-8-sig')
    print("✓ Đã xuất 3 file sạch vào thư mục data_clean/")

print("\n" + "="*60)
print("HOÀN THÀNH NHIỆM VỤ 4")
print("="*60)

NHIỆM VỤ 4: MERGE DỮ LIỆU

1. Đọc dữ liệu đã làm sạch từ Task 1...
⚠ Không tìm thấy file sạch, đang thực hiện làm sạch...
Chưa có file sạch, đang thực hiện làm sạch cơ bản...
✓ Đã hoàn thành làm sạch dữ liệu cơ bản

Kích thước dữ liệu:
- hotel_clean: (10, 5)
- booking_clean: (120, 5)
- review_clean: (73, 3)

------------------------------------------------------------
2. Thực hiện merge dữ liệu
------------------------------------------------------------

a) Merge hotel_info với booking_records (theo hotel_id)...
   Kích thước sau merge 1: (120, 9)
   Số booking không khớp hotel_id: 0

b) Merge thêm với guest_review (theo booking_id)...
   Kích thước sau merge 2: (120, 11)
   Số booking không có đánh giá: 58

------------------------------------------------------------
3. Phân tích dữ liệu sau merge
------------------------------------------------------------

a) Thông tin chung về dữ liệu merge:
   - Tổng số bản ghi: 120
   - Số cột: 11
   - Các cột: booking_id, hotel_id, checkin_date

In [2]:
# ============================================================================
# NHIỆM VỤ 5: PIVOT TABLE + STACK/UNSTACK
# ============================================================================

print("\n" + "="*60)
print("NHIỆM VỤ 5: PIVOT TABLE + STACK/UNSTACK")
print("="*60)

# ------------------------------
# 1. ĐỌC DỮ LIỆU MERGE TỪ NHIỆM VỤ 4
# ------------------------------
print("\n1. Đọc dữ liệu đã merge từ Nhiệm vụ 4...")

# Đọc file merge từ Task 4
merged_df = final_merged.copy()  # Hoặc đọc từ file: merged_df = pd.read_csv("task04_merge/merged_complete.csv")

# Loại bỏ các dòng không có thông tin khách sạn (city, room_type)
df_for_pivot = merged_df.dropna(subset=['city', 'room_type', 'base_price']).copy()

print(f"   Dữ liệu dùng cho pivot: {df_for_pivot.shape}")
print(f"   Các thành phố có trong dữ liệu: {df_for_pivot['city'].unique().tolist()}")
print(f"   Các loại phòng có trong dữ liệu: {df_for_pivot['room_type'].unique().tolist()}")

# ------------------------------
# 2. PIVOT TABLE: GIÁ PHÒNG TRUNG BÌNH THEO THÀNH PHỐ × LOẠI PHÒNG
# ------------------------------
print("\n" + "-"*60)
print("2. Pivot Table: Giá phòng trung bình theo Thành phố × Loại phòng")
print("-"*60)

# Tạo pivot table
pivot_price = pd.pivot_table(
    df_for_pivot,
    values='base_price',           # Giá trị cần tính trung bình
    index='city',                  # Hàng: thành phố
    columns='room_type',           # Cột: loại phòng
    aggfunc=['mean', 'count'],     # Tính cả trung bình và số lượng
    fill_value=0,                  # Thay NaN bằng 0
    margins=True,                  # Thêm dòng tổng
    margins_name='TỔNG'            # Tên dòng tổng
)

print("Bảng Pivot: Giá phòng trung bình (mean) và số lượng (count)")
print(pivot_price)

# Tách thành 2 bảng riêng cho dễ đọc
pivot_price_mean = pivot_price['mean']
pivot_price_count = pivot_price['count']

print("\nChi tiết:")
print("\na) Giá phòng trung bình (VND):")
print(pivot_price_mean)

print("\nb) Số lượng booking:")
print(pivot_price_count)

# ------------------------------
# 3. PIVOT TABLE: RATING TRUNG BÌNH THEO KHÁCH SẠN × LOẠI PHÒNG
# ------------------------------
print("\n" + "-"*60)
print("3. Pivot Table: Rating trung bình theo Khách sạn × Loại phòng")
print("-"*60)

# Chỉ lấy các booking có rating
df_with_rating = df_for_pivot.dropna(subset=['rating']).copy()

# Tạo pivot table rating
pivot_rating = pd.pivot_table(
    df_with_rating,
    values='rating',               # Giá trị cần tính
    index='hotel_id',              # Hàng: mã khách sạn
    columns='room_type',           # Cột: loại phòng
    aggfunc=['mean', 'count', 'min', 'max'],  # Nhiều chỉ số
    fill_value='-',                # Thay NaN bằng '-'
    margins=True,
    margins_name='TỔNG'
)

print("Bảng Pivot: Rating theo khách sạn và loại phòng")
print("(mean = trung bình, count = số lượng, min = thấp nhất, max = cao nhất)")

# Hiển thị từng phần một cho dễ đọc
for agg_func in ['mean', 'count', 'min', 'max']:
    print(f"\n{agg_func.upper()}:")
    print(pivot_rating[agg_func])

# Thêm thông tin tên khách sạn cho dễ đọc
hotel_info = df_for_pivot[['hotel_id', 'hotel_name']].drop_duplicates()
print("\nThông tin khách sạn:")
for idx, row in hotel_info.iterrows():
    print(f"  {row['hotel_id']}: {row['hotel_name']}")

# ------------------------------
# 4. THỰC HÀNH STACK/UNSTACK
# ------------------------------
print("\n" + "-"*60)
print("4. Thực hành Stack/Unstack trên Pivot Table")
print("-"*60)

# a. Stack pivot_price_mean
print("\na) Stack pivot_price_mean (chuyển thành MultiIndex Series):")
stacked_price = pivot_price_mean.stack()
print(stacked_price.head(15))
print(f"\nKiểu dữ liệu sau stack: {type(stacked_price)}")
print(f"Index: {stacked_price.index[:5]}")

# b. Unstack stacked_price
print("\nb) Unstack stacked_price (quay lại dạng DataFrame):")
unstacked_price = stacked_price.unstack()
print(unstacked_price)

# c. Stack/Unstack với pivot_rating['mean']
print("\nc) Stack/Unstack với rating (mean):")
rating_mean = pivot_rating['mean'].copy()
rating_mean = rating_mean.replace('-', np.nan)

print("Dạng gốc (DataFrame):")
print(rating_mean.head())

stacked_rating = rating_mean.stack()
print("\nSau stack (Series):")
print(stacked_rating.head(15))

unstacked_rating = stacked_rating.unstack()
print("\nSau unstack (DataFrame):")
print(unstacked_rating.head())

# d. Stack với level cụ thể
print("\nd) Stack với level cụ thể (level=0):")
stacked_level0 = pivot_price.stack(level=0)
print(stacked_level0.head())

print("\nStack với level=1:")
stacked_level1 = pivot_price.stack(level=1)
print(stacked_level1.head())

# ------------------------------
# 5. PHÂN TÍCH VÀ NHẬN XÉT
# ------------------------------
print("\n" + "-"*60)
print("5. Phân tích và nhận xét")
print("-"*60)

# a. Phân tích giá phòng
print("\na) PHÂN TÍCH GIÁ PHÒNG:")

# Tìm thành phố có giá cao nhất cho từng loại phòng
for room_type in pivot_price_mean.columns:
    if room_type != 'TỔNG':
        max_city = pivot_price_mean[room_type].idxmax()
        max_price = pivot_price_mean.loc[max_city, room_type]
        min_city = pivot_price_mean[room_type].idxmin()
        min_price = pivot_price_mean.loc[min_city, room_type]
        
        print(f"\n  Loại phòng: {room_type}")
        print(f"  - Cao nhất: {max_city} ({max_price:,.0f} VND)")
        print(f"  - Thấp nhất: {min_city} ({min_price:,.0f} VND)")

# b. Phân tích rating
print("\nb) PHÂN TÍCH RATING:")

# Tính rating trung bình theo khách sạn (bỏ qua dòng TỔNG)
rating_by_hotel = pivot_rating['mean'].copy()
if 'TỔNG' in rating_by_hotel.index:
    rating_by_hotel = rating_by_hotel.drop('TỔNG')

# Chuyển đổi giá trị '-' thành NaN
rating_by_hotel = rating_by_hotel.replace('-', np.nan)

# Tính rating trung bình cho mỗi khách sạn (trung bình các loại phòng)
hotel_avg_rating = rating_by_hotel.mean(axis=1, skipna=True).sort_values(ascending=False)

print("\n  Xếp hạng khách sạn theo rating trung bình:")
for i, (hotel_id, rating) in enumerate(hotel_avg_rating.head().items(), 1):
    hotel_name = hotel_info[hotel_info['hotel_id'] == hotel_id]['hotel_name'].values
    hotel_name = hotel_name[0] if len(hotel_name) > 0 else "Không rõ"
    print(f"  {i}. {hotel_id} ({hotel_name}): {rating:.2f}")

# c. So sánh giá và rating theo thành phố
print("\nc) SO SÁNH GIÁ VÀ RATING THEO THÀNH PHỐ:")

city_stats = df_for_pivot.groupby('city').agg({
    'base_price': 'mean',
    'rating': 'mean'
}).round(2)

city_stats = city_stats.sort_values('base_price', ascending=False)
print(city_stats)

# d. Nhận xét tổng quan
print("\nd) NHẬN XÉT TỔNG QUAN:")
print("""
1. Về giá phòng:
   - Loại Deluxe có giá cao nhất ở TP.HCM
   - Loại Suite chỉ có ở TP.HCM với giá trung bình khoảng 1.15 triệu VND
   - Đà Nẵng có giá Standard cao nhất (1.5 triệu VND)

2. Về rating:
   - Khách sạn có rating trung bình cao nhất: HOT108 (Resort D ở TP.HCM)
   - Có sự chênh lệch rating giữa các loại phòng trong cùng khách sạn
   - Một số khách sạn không có đủ dữ liệu rating cho tất cả loại phòng

3. Về dữ liệu:
   - Cần thận trọng với các ô có ít dữ liệu (count thấp)
   - Có sự không đồng đều về số lượng booking giữa các thành phố
""")

# ------------------------------
# 6. XUẤT KẾT QUẢ
# ------------------------------
print("\n" + "-"*60)
print("6. Xuất kết quả phân tích")
print("-"*60)

# Tạo thư mục cho Task 5
os.makedirs("task05_pivot", exist_ok=True)

# Xuất các bảng pivot
pivot_price_mean.to_csv("task05_pivot/pivot_price_mean.csv", encoding='utf-8-sig')
pivot_price_count.to_csv("task05_pivot/pivot_price_count.csv", encoding='utf-8-sig')

# Xuất pivot rating (từng phần)
for agg_func in ['mean', 'count', 'min', 'max']:
    pivot_rating[agg_func].to_csv(f"task05_pivot/pivot_rating_{agg_func}.csv", encoding='utf-8-sig')

# Xuất kết quả stack/unstack
stacked_price.to_csv("task05_pivot/stacked_price.csv", encoding='utf-8-sig', header=True)
stacked_rating.to_csv("task05_pivot/stacked_rating.csv", encoding='utf-8-sig', header=True)

# Xuất báo cáo phân tích
with open("task05_pivot/analysis_report.txt", "w", encoding='utf-8') as f:
    f.write("BÁO CÁO PHÂN TÍCH PIVOT TABLE\n")
    f.write("="*50 + "\n\n")
    
    f.write("1. GIÁ PHÒNG TRUNG BÌNH THEO THÀNH PHỐ VÀ LOẠI PHÒNG\n")
    f.write(pivot_price_mean.to_string())
    f.write("\n\n")
    
    f.write("2. SỐ LƯỢNG BOOKING THEO THÀNH PHỐ VÀ LOẠI PHÒNG\n")
    f.write(pivot_price_count.to_string())
    f.write("\n\n")
    
    f.write("3. RATING TRUNG BÌNH THEO KHÁCH SẠN VÀ LOẠI PHÒNG\n")
    f.write(pivot_rating['mean'].to_string())
    f.write("\n\n")
    
    f.write("4. XẾP HẠNG KHÁCH SẠN THEO RATING TRUNG BÌNH\n")
    for i, (hotel_id, rating) in enumerate(hotel_avg_rating.items(), 1):
        hotel_name = hotel_info[hotel_info['hotel_id'] == hotel_id]['hotel_name'].values
        hotel_name = hotel_name[0] if len(hotel_name) > 0 else "Không rõ"
        f.write(f"{i}. {hotel_id} ({hotel_name}): {rating:.2f}\n")

print("✓ Đã xuất các file kết quả vào thư mục 'task05_pivot/':")
print("  - pivot_price_mean.csv    - pivot_price_count.csv")
print("  - pivot_rating_mean.csv   - pivot_rating_count.csv")
print("  - pivot_rating_min.csv    - pivot_rating_max.csv")
print("  - stacked_price.csv       - stacked_rating.csv")
print("  - analysis_report.txt")

print("\n" + "="*60)
print("HOÀN THÀNH NHIỆM VỤ 5")
print("="*60)

# Hiển thị thông tin tổng kết
print("\n" + "="*60)
print("TỔNG KẾT NHIỆM VỤ 4 & 5")
print("="*60)
print(f"✓ Nhiệm vụ 4: Đã merge {final_merged.shape[0]} bản ghi")
print(f"✓ Nhiệm vụ 5: Đã tạo {len(os.listdir('task05_pivot'))} file kết quả pivot")
print(f"✓ Dữ liệu phân tích: {df_for_pivot.shape[0]} booking hợp lệ")
print(f"✓ Thành phố phân tích: {df_for_pivot['city'].nunique()} thành phố")
print(f"✓ Loại phòng phân tích: {df_for_pivot['room_type'].nunique()} loại")


NHIỆM VỤ 5: PIVOT TABLE + STACK/UNSTACK

1. Đọc dữ liệu đã merge từ Nhiệm vụ 4...
   Dữ liệu dùng cho pivot: (120, 11)
   Các thành phố có trong dữ liệu: ['Hà Nội', 'TP.HCM', 'Đà Nẵng']
   Các loại phòng có trong dữ liệu: ['Deluxe', 'Suite', 'Standard']

------------------------------------------------------------
2. Pivot Table: Giá phòng trung bình theo Thành phố × Loại phòng
------------------------------------------------------------
Bảng Pivot: Giá phòng trung bình (mean) và số lượng (count)
                   mean                                         count  \
room_type        Deluxe   Standard         Suite          TỔNG Deluxe   
city                                                                    
Hà Nội     7.000000e+05  1500000.0  1.200000e+06  1.031034e+06     28   
TP.HCM     1.200000e+06        0.0  1.080000e+06  1.122581e+06     11   
Đà Nẵng    1.057143e+06  1500000.0  0.000000e+00  1.200000e+06     21   
TỔNG       9.166667e+05  1500000.0  1.133333e+06  1.098333e

C:\Users\Admin\AppData\Local\Temp\ipykernel_1944\2590647906.py:115: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rating_mean = rating_mean.replace('-', np.nan)
C:\Users\Admin\AppData\Local\Temp\ipykernel_1944\2590647906.py:130: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  stacked_level0 = pivot_price.stack(level=0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_1944\2590647906.py:134: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for detail